In [1]:
import sys
print("Python executable:", sys.executable)

import langchain
import langgraph
import sqlalchemy
import pinecone
import sentence_transformers

print("All core imports successful ✅")

Python executable: c:\Users\GCV\dev\work\agentic-gst-ai\gstagent\Scripts\python.exe
All core imports successful ✅


In [2]:
from sqlalchemy import create_engine, Column, Integer, String, Float
from sqlalchemy.orm import declarative_base, sessionmaker
import os

In [3]:
# Create a local SQLite database file
DB_PATH = "invoices.db"
DATABASE_URL = f"sqlite:///{DB_PATH}"

engine = create_engine(DATABASE_URL, echo=False)
SessionLocal = sessionmaker(bind=engine)

print("SQLite engine created at:", os.path.abspath(DB_PATH))


SQLite engine created at: c:\Users\GCV\dev\work\agentic-gst-ai\notebooks\invoices.db


In [4]:
Base = declarative_base()

class Invoice(Base):
    __tablename__ = "invoices"

    id = Column(Integer, primary_key=True, index=True)
    invoice_date = Column(String, nullable=False)
    location = Column(String, nullable=False)
    vendor = Column(String, nullable=False)
    item = Column(String, nullable=False)
    quantity = Column(Integer, nullable=False)
    amount = Column(Float, nullable=False)
    gst_rate = Column(Float, nullable=False)

    def __repr__(self):
        return (
            f"<Invoice(id={self.id}, date={self.invoice_date}, "
            f"vendor={self.vendor}, item={self.item}, amount={self.amount})>"
        )


In [5]:
# Create tables in the database
Base.metadata.create_all(bind=engine)

print("Invoices table created successfully ✅")


Invoices table created successfully ✅


In [6]:
# Manual sanity check insert
session = SessionLocal()

test_invoice = Invoice(
    invoice_date="2025-01-05",
    location="Bangalore",
    vendor="Amazon",
    item="Laptop",
    quantity=1,
    amount=65000.0,
    gst_rate=18.0
)

session.add(test_invoice)
session.commit()

print("Test invoice inserted with ID:", test_invoice.id)
session.close()


Test invoice inserted with ID: 13


In [7]:
# Read back data
session = SessionLocal()
invoices = session.query(Invoice).all()

for inv in invoices:
    print(inv)

session.close()

<Invoice(id=1, date=2025-01-05, vendor=Amazon, item=Laptop, amount=65000.0)>
<Invoice(id=2, date=2025-01-10, vendor=Flipkart, item=Headphones, amount=4000.0)>
<Invoice(id=3, date=2025-01-05, vendor=Amazon, item=Laptop, amount=65000.0)>
<Invoice(id=4, date=2025-01-05, vendor=Flipkart, item=Headphones, amount=4000.0)>
<Invoice(id=5, date=2025-01-10, vendor=Reliance, item=Office Chair, amount=9000.0)>
<Invoice(id=6, date=2025-01-12, vendor=Croma, item=Smartphone, amount=30000.0)>
<Invoice(id=7, date=2025-01-15, vendor=IKEA, item=Study Table, amount=12000.0)>
<Invoice(id=8, date=2025-01-15, vendor=DMart, item=Groceries, amount=3500.0)>
<Invoice(id=9, date=2025-01-05, vendor=Amazon, item=Laptop, amount=65000.0)>
<Invoice(id=10, date=2025-01-20, vendor=Amazon, item=Mouse, amount=1200.0)>
<Invoice(id=11, date=2025-01-05, vendor=Amazon, item=Laptop, amount=65000.0)>
<Invoice(id=12, date=2025-01-05, vendor=Amazon, item=Laptop, amount=65000.0)>
<Invoice(id=13, date=2025-01-05, vendor=Amazon, ite

In [8]:
from sqlalchemy import text

In [9]:
from sqlalchemy import text
from sqlalchemy.orm import sessionmaker

SessionLocal = sessionmaker(bind=engine)

def run_sql_tool(sql_query: str) -> str:
    """
    Executes SELECT or INSERT SQL safely and returns LLM-friendly output.
    """

    session = SessionLocal()   # ✅ session ALWAYS created

    try:
        if sql_query.lower().startswith("select"):
            result = session.execute(text(sql_query))
            rows = result.fetchall()

            if not rows:
                return "No records found."

            # Convert rows to dict-like output
            columns = result.keys()
            output = [
                dict(zip(columns, row))
                for row in rows
            ]

            return str(output)

        elif sql_query.lower().startswith("insert"):
            session.execute(text(sql_query))
            session.commit()
            return "Insert executed successfully."

        else:
            return "Only SELECT and INSERT queries are allowed."

    except Exception as e:
        session.rollback()
        return f"SQL TOOL ERROR: {str(e)}"

    finally:
        session.close()   # ✅ always closes


In [10]:
# Test SELECT via tool
query = "SELECT * FROM invoices;"
print(run_sql_tool(query))

[{'id': 1, 'invoice_date': '2025-01-05', 'location': 'Bangalore', 'vendor': 'Amazon', 'item': 'Laptop', 'quantity': 1, 'amount': 65000.0, 'gst_rate': 18.0}, {'id': 2, 'invoice_date': '2025-01-10', 'location': 'Chennai', 'vendor': 'Flipkart', 'item': 'Headphones', 'quantity': 2, 'amount': 4000.0, 'gst_rate': 18.0}, {'id': 3, 'invoice_date': '2025-01-05', 'location': 'Bangalore', 'vendor': 'Amazon', 'item': 'Laptop', 'quantity': 1, 'amount': 65000.0, 'gst_rate': 18.0}, {'id': 4, 'invoice_date': '2025-01-05', 'location': 'Chennai', 'vendor': 'Flipkart', 'item': 'Headphones', 'quantity': 2, 'amount': 4000.0, 'gst_rate': 18.0}, {'id': 5, 'invoice_date': '2025-01-10', 'location': 'Kochi', 'vendor': 'Reliance', 'item': 'Office Chair', 'quantity': 1, 'amount': 9000.0, 'gst_rate': 12.0}, {'id': 6, 'invoice_date': '2025-01-12', 'location': 'Bangalore', 'vendor': 'Croma', 'item': 'Smartphone', 'quantity': 1, 'amount': 30000.0, 'gst_rate': 18.0}, {'id': 7, 'invoice_date': '2025-01-15', 'location':

In [11]:
# Test INSERT via tool
insert_query = """
INSERT INTO invoices (invoice_date, location, vendor, item, quantity, amount, gst_rate)
VALUES ('2025-01-10', 'Chennai', 'Flipkart', 'Headphones', 2, 4000, 18);
"""

print(run_sql_tool(insert_query))


Only SELECT and INSERT queries are allowed.


In [12]:
print(run_sql_tool("SELECT * FROM invoices;"))

[{'id': 1, 'invoice_date': '2025-01-05', 'location': 'Bangalore', 'vendor': 'Amazon', 'item': 'Laptop', 'quantity': 1, 'amount': 65000.0, 'gst_rate': 18.0}, {'id': 2, 'invoice_date': '2025-01-10', 'location': 'Chennai', 'vendor': 'Flipkart', 'item': 'Headphones', 'quantity': 2, 'amount': 4000.0, 'gst_rate': 18.0}, {'id': 3, 'invoice_date': '2025-01-05', 'location': 'Bangalore', 'vendor': 'Amazon', 'item': 'Laptop', 'quantity': 1, 'amount': 65000.0, 'gst_rate': 18.0}, {'id': 4, 'invoice_date': '2025-01-05', 'location': 'Chennai', 'vendor': 'Flipkart', 'item': 'Headphones', 'quantity': 2, 'amount': 4000.0, 'gst_rate': 18.0}, {'id': 5, 'invoice_date': '2025-01-10', 'location': 'Kochi', 'vendor': 'Reliance', 'item': 'Office Chair', 'quantity': 1, 'amount': 9000.0, 'gst_rate': 12.0}, {'id': 6, 'invoice_date': '2025-01-12', 'location': 'Bangalore', 'vendor': 'Croma', 'item': 'Smartphone', 'quantity': 1, 'amount': 30000.0, 'gst_rate': 18.0}, {'id': 7, 'invoice_date': '2025-01-15', 'location':

In [13]:
seed_queries = [
    """
    INSERT INTO invoices (invoice_date, location, vendor, item, quantity, amount, gst_rate)
    VALUES ('2025-01-05', 'Bangalore', 'Amazon', 'Laptop', 1, 65000, 18);
    """,

    """
    INSERT INTO invoices (invoice_date, location, vendor, item, quantity, amount, gst_rate)
    VALUES ('2025-01-05', 'Chennai', 'Flipkart', 'Headphones', 2, 4000, 18);
    """,

    """
    INSERT INTO invoices (invoice_date, location, vendor, item, quantity, amount, gst_rate)
    VALUES ('2025-01-10', 'Kochi', 'Reliance', 'Office Chair', 1, 9000, 12);
    """,

    """
    INSERT INTO invoices (invoice_date, location, vendor, item, quantity, amount, gst_rate)
    VALUES ('2025-01-12', 'Bangalore', 'Croma', 'Smartphone', 1, 30000, 18);
    """,

    """
    INSERT INTO invoices (invoice_date, location, vendor, item, quantity, amount, gst_rate)
    VALUES ('2025-01-15', 'Hyderabad', 'IKEA', 'Study Table', 1, 12000, 12);
    """,

    """
    INSERT INTO invoices (invoice_date, location, vendor, item, quantity, amount, gst_rate)
    VALUES ('2025-01-15', 'Mumbai', 'DMart', 'Groceries', 5, 3500, 5);
    """
]

for q in seed_queries:
    print(run_sql_tool(q))


Only SELECT and INSERT queries are allowed.
Only SELECT and INSERT queries are allowed.
Only SELECT and INSERT queries are allowed.
Only SELECT and INSERT queries are allowed.
Only SELECT and INSERT queries are allowed.
Only SELECT and INSERT queries are allowed.


In [14]:
print(run_sql_tool("SELECT * FROM invoices;"))


[{'id': 1, 'invoice_date': '2025-01-05', 'location': 'Bangalore', 'vendor': 'Amazon', 'item': 'Laptop', 'quantity': 1, 'amount': 65000.0, 'gst_rate': 18.0}, {'id': 2, 'invoice_date': '2025-01-10', 'location': 'Chennai', 'vendor': 'Flipkart', 'item': 'Headphones', 'quantity': 2, 'amount': 4000.0, 'gst_rate': 18.0}, {'id': 3, 'invoice_date': '2025-01-05', 'location': 'Bangalore', 'vendor': 'Amazon', 'item': 'Laptop', 'quantity': 1, 'amount': 65000.0, 'gst_rate': 18.0}, {'id': 4, 'invoice_date': '2025-01-05', 'location': 'Chennai', 'vendor': 'Flipkart', 'item': 'Headphones', 'quantity': 2, 'amount': 4000.0, 'gst_rate': 18.0}, {'id': 5, 'invoice_date': '2025-01-10', 'location': 'Kochi', 'vendor': 'Reliance', 'item': 'Office Chair', 'quantity': 1, 'amount': 9000.0, 'gst_rate': 12.0}, {'id': 6, 'invoice_date': '2025-01-12', 'location': 'Bangalore', 'vendor': 'Croma', 'item': 'Smartphone', 'quantity': 1, 'amount': 30000.0, 'gst_rate': 18.0}, {'id': 7, 'invoice_date': '2025-01-15', 'location':

In [15]:
print(run_sql_tool(
    "SELECT vendor, item, amount FROM invoices WHERE invoice_date = '2025-01-05';"
))


[{'vendor': 'Amazon', 'item': 'Laptop', 'amount': 65000.0}, {'vendor': 'Amazon', 'item': 'Laptop', 'amount': 65000.0}, {'vendor': 'Flipkart', 'item': 'Headphones', 'amount': 4000.0}, {'vendor': 'Amazon', 'item': 'Laptop', 'amount': 65000.0}, {'vendor': 'Amazon', 'item': 'Laptop', 'amount': 65000.0}, {'vendor': 'Amazon', 'item': 'Laptop', 'amount': 65000.0}, {'vendor': 'Amazon', 'item': 'Laptop', 'amount': 65000.0}]


In [16]:
print(run_sql_tool(
    "SELECT invoice_date, vendor, item FROM invoices WHERE location = 'Bangalore';"
))


[{'invoice_date': '2025-01-05', 'vendor': 'Amazon', 'item': 'Laptop'}, {'invoice_date': '2025-01-05', 'vendor': 'Amazon', 'item': 'Laptop'}, {'invoice_date': '2025-01-12', 'vendor': 'Croma', 'item': 'Smartphone'}, {'invoice_date': '2025-01-05', 'vendor': 'Amazon', 'item': 'Laptop'}, {'invoice_date': '2025-01-20', 'vendor': 'Amazon', 'item': 'Mouse'}, {'invoice_date': '2025-01-05', 'vendor': 'Amazon', 'item': 'Laptop'}, {'invoice_date': '2025-01-05', 'vendor': 'Amazon', 'item': 'Laptop'}, {'invoice_date': '2025-01-05', 'vendor': 'Amazon', 'item': 'Laptop'}]


In [17]:
print(run_sql_tool(
    "SELECT vendor, item, gst_rate FROM invoices WHERE gst_rate = 12;"
))


[{'vendor': 'Reliance', 'item': 'Office Chair', 'gst_rate': 12.0}, {'vendor': 'IKEA', 'item': 'Study Table', 'gst_rate': 12.0}]


In [18]:
import os
from dotenv import load_dotenv

from sentence_transformers import SentenceTransformer
import pinecone


In [50]:
load_dotenv()

PINECONE_API_KEY = os.getenv("PINECONE_API_KEY")
PINECONE_ENV = os.getenv("PINECONE_ENV")

assert PINECONE_API_KEY is not None, "PINECONE_API_KEY not set"
assert PINECONE_ENV is not None, "PINECONE_ENV not set"

print("Pinecone environment loaded ✅")


Pinecone environment loaded ✅


In [20]:
pc = pinecone.Pinecone(api_key=PINECONE_API_KEY)
print("Pinecone client initialized ✅")

Pinecone client initialized ✅


In [21]:
embedding_model = SentenceTransformer("all-MiniLM-L6-v2")
EMBEDDING_DIM = embedding_model.get_sentence_embedding_dimension()

print("Embedding dimension:", EMBEDDING_DIM)

Loading weights:   0%|          | 0/103 [00:00<?, ?it/s]

BertModel LOAD REPORT from: sentence-transformers/all-MiniLM-L6-v2
Key                     | Status     |  | 
------------------------+------------+--+-
embeddings.position_ids | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.


Embedding dimension: 384


In [22]:
from pinecone import ServerlessSpec

INDEX_NAME = "gst-rules-index"

existing_indexes = pc.list_indexes().names()

if INDEX_NAME not in existing_indexes:
    pc.create_index(
        name=INDEX_NAME,
        dimension=EMBEDDING_DIM,
        metric="cosine",
        spec=ServerlessSpec(
            cloud="aws",
            region="us-east-1"
        )
    )
    print("Pinecone index created ✅")
else:
    print("Pinecone index already exists ✅")


Pinecone index already exists ✅


In [23]:
index = pc.Index(INDEX_NAME)
print("Connected to Pinecone index ✅")

Connected to Pinecone index ✅


In [24]:
GST_DOC_PATH = "../data/gst_rules.txt"

with open(GST_DOC_PATH, "r", encoding="utf-8") as f:
    gst_text = f.read()

# Simple line-based chunking
gst_chunks = [line.strip() for line in gst_text.split("\n") if line.strip()]

print("GST chunks loaded:", len(gst_chunks))
gst_chunks


GST chunks loaded: 22


['Electronics such as laptops, smartphones, tablets, and accessories attract 18 percent GST.',
 'Large home appliances including refrigerators, washing machines, and air conditioners attract 18 percent GST.',
 'Furniture items such as tables, chairs, beds, and cupboards attract 12 percent GST.',
 'Office furniture and modular workstations attract 12 percent GST.',
 'Essential goods such as groceries, cereals, rice, wheat, milk, and vegetables attract 5 percent GST.',
 'Packaged food items including branded snacks and processed foods attract 12 percent GST.',
 'Luxury items such as premium cars, high-end motorcycles, and luxury watches attract 28 percent GST.',
 'Clothing priced below 1000 rupees per item attracts 5 percent GST.',
 'Clothing priced above 1000 rupees per item attracts 12 percent GST.',
 'Services such as consulting, professional services, IT services, and software services attract 18 percent GST.',
 'Hotel accommodation priced below 1000 rupees per day is exempt from GST

In [25]:
vectors = []

for i, chunk in enumerate(gst_chunks):
    embedding = embedding_model.encode(
        chunk, normalize_embeddings=True
    ).tolist()

    vectors.append(
        (
            f"gst-{i}",
            embedding,
            {"text": chunk}
        )
    )

index.upsert(vectors)
print("GST rules embedded and stored in Pinecone ✅")


GST rules embedded and stored in Pinecone ✅


In [26]:
query = "What is the GST rate for electronics?"

query_embedding = embedding_model.encode(
    query, normalize_embeddings=True
).tolist()

results = index.query(
    vector=query_embedding,
    top_k=3,
    include_metadata=True
)

for match in results["matches"]:
    print(match["metadata"]["text"])


Electronics such as laptops, smartphones, tablets, and accessories attract 18 percent GST.
Services such as consulting, professional services, IT services, and software services attract 18 percent GST.
Large home appliances including refrigerators, washing machines, and air conditioners attract 18 percent GST.


In [27]:
def gst_rag_agent(query: str, top_k: int = 3) -> str:
    """
    Retrieves relevant GST rules using semantic search.
    This agent has NO access to SQL or invoice data.
    """

    try:
        # Embed query
        query_embedding = embedding_model.encode(
            query, normalize_embeddings=True
        ).tolist()

        # Query Pinecone
        results = index.query(
            vector=query_embedding,
            top_k=top_k,
            include_metadata=True
        )

        if not results["matches"]:
            return "No relevant GST rules found."

        # Collect GST rules
        gst_rules = []
        for match in results["matches"]:
            rule_text = match["metadata"].get("text")
            if rule_text:
                gst_rules.append(rule_text)

        # Return as readable block
        return "\n".join(gst_rules)

    except Exception as e:
        return f"GST RAG ERROR: {str(e)}"


In [28]:
print(
    gst_rag_agent(
        "What is the GST rate for electronics like laptops?"
    )
)

Electronics such as laptops, smartphones, tablets, and accessories attract 18 percent GST.
Services such as consulting, professional services, IT services, and software services attract 18 percent GST.
GST is calculated on the transaction value, including discounts applied at the time of sale.


In [29]:
print(
    gst_rag_agent(
        "Is GST split into CGST and SGST?"
    )
)


For intra-state transactions, GST is split into Central GST (CGST) and State GST (SGST).
For inter-state transactions, Integrated GST (IGST) is applicable instead of CGST and SGST.
GST is calculated on the transaction value, including discounts applied at the time of sale.


In [51]:
from langchain_google_genai import ChatGoogleGenerativeAI

# Single shared LLM instance (used by all agents)
llm = ChatGoogleGenerativeAI(
    model="gemini-flash-latest",
    temperature=0.1,
    max_output_tokens=1024
)

print("Gemini Flash LLM initialized ✅")


Gemini Flash LLM initialized ✅


In [52]:
response = llm.invoke("Say OK if you are working.")
print(response.content)

OK


In [32]:
SQL_AGENT_SYSTEM_PROMPT = """
You are an expert SQL assistant.

You have access to a SQLite database with ONE table:

Table name: invoices

Columns:
- id (integer, primary key)
- invoice_date (string, format YYYY-MM-DD)
- location (string)
- vendor (string)
- item (string)
- quantity (integer)
- amount (float)
- gst_rate (float)

Rules:
- Generate ONLY valid SQL.
- Use SELECT for reading data.
- Use INSERT for adding new records.
- Do NOT explain the SQL.
- Do NOT include markdown.
- Do NOT include backticks.
- Return ONLY the SQL query.
"""


In [33]:
def sql_agent(user_query: str) -> str:
    """
    Converts natural language into SQL, executes it using run_sql_tool,
    and returns the result.
    """

    try:
        # Step 1: Ask LLM to generate SQL
        prompt = f"""
{SQL_AGENT_SYSTEM_PROMPT}

User request:
{user_query}
"""

        sql_response = llm.invoke(prompt)
        sql_query = sql_response.content.strip()

        # Basic safety check
        if not sql_query.lower().startswith(("select", "insert")):
            return "SQL AGENT ERROR: Only SELECT or INSERT queries are allowed."

        # Step 2: Execute SQL via tool
        result = run_sql_tool(sql_query)

        # Step 3: Return result
        return result

    except Exception as e:
        return f"SQL AGENT ERROR: {str(e)}"


In [34]:
print(
    sql_agent(
        "Show all purchases made on 2025-01-05"
    )
)


[{'id': 1, 'invoice_date': '2025-01-05', 'location': 'Bangalore', 'vendor': 'Amazon', 'item': 'Laptop', 'quantity': 1, 'amount': 65000.0, 'gst_rate': 18.0}, {'id': 3, 'invoice_date': '2025-01-05', 'location': 'Bangalore', 'vendor': 'Amazon', 'item': 'Laptop', 'quantity': 1, 'amount': 65000.0, 'gst_rate': 18.0}, {'id': 4, 'invoice_date': '2025-01-05', 'location': 'Chennai', 'vendor': 'Flipkart', 'item': 'Headphones', 'quantity': 2, 'amount': 4000.0, 'gst_rate': 18.0}, {'id': 9, 'invoice_date': '2025-01-05', 'location': 'Bangalore', 'vendor': 'Amazon', 'item': 'Laptop', 'quantity': 1, 'amount': 65000.0, 'gst_rate': 18.0}, {'id': 11, 'invoice_date': '2025-01-05', 'location': 'Bangalore', 'vendor': 'Amazon', 'item': 'Laptop', 'quantity': 1, 'amount': 65000.0, 'gst_rate': 18.0}, {'id': 12, 'invoice_date': '2025-01-05', 'location': 'Bangalore', 'vendor': 'Amazon', 'item': 'Laptop', 'quantity': 1, 'amount': 65000.0, 'gst_rate': 18.0}, {'id': 13, 'invoice_date': '2025-01-05', 'location': 'Bang

In [35]:
print(
    sql_agent(
        "Add a purchase on 2025-01-20 in Bangalore from Amazon for a Mouse costing 1200 with GST rate 18 percent"
    )
)


Insert executed successfully.


In [36]:
print(
    sql_agent(
        "Show all purchases from Amazon"
    )
)


[{'id': 1, 'invoice_date': '2025-01-05', 'location': 'Bangalore', 'vendor': 'Amazon', 'item': 'Laptop', 'quantity': 1, 'amount': 65000.0, 'gst_rate': 18.0}, {'id': 3, 'invoice_date': '2025-01-05', 'location': 'Bangalore', 'vendor': 'Amazon', 'item': 'Laptop', 'quantity': 1, 'amount': 65000.0, 'gst_rate': 18.0}, {'id': 9, 'invoice_date': '2025-01-05', 'location': 'Bangalore', 'vendor': 'Amazon', 'item': 'Laptop', 'quantity': 1, 'amount': 65000.0, 'gst_rate': 18.0}, {'id': 10, 'invoice_date': '2025-01-20', 'location': 'Bangalore', 'vendor': 'Amazon', 'item': 'Mouse', 'quantity': 1, 'amount': 1200.0, 'gst_rate': 0.18}, {'id': 11, 'invoice_date': '2025-01-05', 'location': 'Bangalore', 'vendor': 'Amazon', 'item': 'Laptop', 'quantity': 1, 'amount': 65000.0, 'gst_rate': 18.0}, {'id': 12, 'invoice_date': '2025-01-05', 'location': 'Bangalore', 'vendor': 'Amazon', 'item': 'Laptop', 'quantity': 1, 'amount': 65000.0, 'gst_rate': 18.0}, {'id': 13, 'invoice_date': '2025-01-05', 'location': 'Bangalor

In [54]:
SUPERVISOR_SYSTEM_PROMPT = """
You are a Supervisor agent that plans actions for an agentic system.

Your job is to decide WHICH actions should be executed to answer the user's query.
You do NOT answer the question yourself.

Available actions:
- SQL_AGENT: fetch or insert invoice / purchase data from SQL database
- GST_AGENT: retrieve GST rules, rates, or tax logic
- REASONING: generate the final natural language answer

Rules:
- If data from database is needed, include SQL_AGENT
- If GST knowledge is needed, include GST_AGENT
- If the user expects an answer, REASONING MUST be the final action
- Output ONLY valid JSON
- No explanations, no extra text

JSON format:
{
  "actions": [
    {"type": "<ACTION_TYPE>", "input": "<instruction>"}
  ]
}

Examples:

User: What is the GST rate for electronics?
Output:
{
  "actions": [
    {"type": "GST_AGENT", "input": "What is the GST rate for electronics?"},
    {"type": "REASONING", "input": "Answer the question using GST rules"}
  ]
}

User: Show all purchases made on 2025-01-05
Output:
{
  "actions": [
    {"type": "SQL_AGENT", "input": "Show all purchases made on 2025-01-05"},
    {"type": "REASONING", "input": "Summarize the SQL results"}
  ]
}

User: Calculate GST for purchases made on 2025-01-05
Output:
{
  "actions": [
    {"type": "SQL_AGENT", "input": "Fetch purchases made on 2025-01-05"},
    {"type": "GST_AGENT", "input": "GST rates applicable to these items"},
    {"type": "REASONING", "input": "Calculate GST and explain clearly"}
  ]
}
"""


In [57]:
import json
import re

def supervisor_agent(user_input: str, conversation_history: list):
    prompt = f"""
SYSTEM:
{SUPERVISOR_SYSTEM_PROMPT}

Conversation history:
{conversation_history}

User input:
{user_input}
"""

    response = llm.invoke(prompt)

    # Gemini-safe extraction
    if isinstance(response.content, list):
        raw_text = response.content[0].get("text", "").strip()
    else:
        raw_text = str(response.content).strip()

    # Strip markdown fences if Gemini adds them
    raw_text = re.sub(r"^```(?:json)?", "", raw_text, flags=re.IGNORECASE).strip()
    raw_text = re.sub(r"```$", "", raw_text).strip()

    return json.loads(raw_text)


In [58]:
print(
    supervisor_agent(
        "Show all purchases made on 2025-01-05",
        conversation_history=[]
    )
)


{'actions': [{'type': 'SQL_AGENT', 'input': 'Show all purchases made on 2025-01-05'}, {'type': 'REASONING', 'input': 'Summarize the SQL results'}]}


In [40]:
print(
    supervisor_agent(
        "What is the GST rate for electronics?",
        conversation_history=[]
    )
)


{'actions': [{'type': 'GST_AGENT', 'input': 'What is the GST rate for electronics?'}, {'type': 'REASONING', 'input': 'Provide the GST rate for electronics.'}]}


In [59]:
from typing import TypedDict, List, Dict, Any
from langgraph.graph import StateGraph, END


In [60]:
class AgentState(TypedDict):
    user_input: str
    conversation_history: list

    supervisor_plan: Dict[str, Any]

    sql_result: str
    gst_result: str

    final_answer: str


In [61]:
def supervisor_node(state: AgentState) -> AgentState:
    plan = supervisor_agent(
        state["user_input"],
        state["conversation_history"]
    )
    state["supervisor_plan"] = plan
    return state


In [62]:
def sql_node(state: AgentState) -> AgentState:
    action = state["supervisor_plan"]["actions"][0]
    result = sql_agent(action["input"])
    state["sql_result"] = result
    return state


In [63]:
def gst_node(state: AgentState) -> AgentState:
    action = state["supervisor_plan"]["actions"][0]
    result = gst_rag_agent(action["input"])
    state["gst_result"] = result
    return state


In [64]:
def reasoning_node(state: AgentState) -> AgentState:
    prompt = f"""
User question:
{state["user_input"]}

SQL Result:
{state.get("sql_result", "")}

GST Rules:
{state.get("gst_result", "")}

Answer clearly and concisely.
"""

    response = llm.invoke(prompt)

    if isinstance(response.content, list):
        state["final_answer"] = response.content[0].get("text", "")
    else:
        state["final_answer"] = response.content

    return state


In [66]:
def route_from_supervisor(state: AgentState) -> str:
    actions = state["supervisor_plan"]["actions"]
    first_action = actions[0]["type"]

    if first_action == "SQL_AGENT":
        return "sql"
    elif first_action == "GST_AGENT":
        return "gst"
    else:
        return "reasoning"


In [67]:
graph = StateGraph(AgentState)

# Nodes
graph.add_node("supervisor", supervisor_node)
graph.add_node("sql", sql_node)
graph.add_node("gst", gst_node)
graph.add_node("reasoning", reasoning_node)

# Entry point
graph.set_entry_point("supervisor")

# Conditional routing
graph.add_conditional_edges(
    "supervisor",
    route_from_supervisor,
    {
        "sql": "sql",
        "gst": "gst",
        "reasoning": "reasoning"
    }
)

# Flow into reasoning
graph.add_edge("sql", "reasoning")
graph.add_edge("gst", "reasoning")

# End
graph.add_edge("reasoning", END)

compiled_graph = graph.compile()

print("LangGraph compiled successfully ✅")


LangGraph compiled successfully ✅


In [68]:
result = compiled_graph.invoke({
    "user_input": "What is the GST rate for electronics?",
    "conversation_history": [],
    "supervisor_plan": {},
    "sql_result": "",
    "gst_result": "",
    "final_answer": ""
})

print(result["final_answer"])


The GST rate for electronics such as laptops, smartphones, tablets, and accessories is **18 percent**.


In [69]:
result = compiled_graph.invoke({
    "user_input": "Show all purchases made on 2025-01-05",
    "conversation_history": [],
    "supervisor_plan": {},
    "sql_result": "",
    "gst_result": "",
    "final_answer": ""
})

print(result["final_answer"])


The following purchases were made on **2025-01-05**:

| ID | Invoice Date | Location | Vendor | Item | Quantity | Amount (INR) | GST Rate (%) |
|----|--------------|----------|----------|------------|----------|--------------|--------------|
| 1  | 2025-01-05   | Bangalore| Amazon   | Laptop     | 1        | 65000.0      | 18.0         |
| 3  | 2025-01-05   | Bangalore| Amazon   | Laptop     | 1        | 65000.0      | 18.0         |
| 4  | 2025-01-05   | Chennai  | Flipkart | Headphones | 2        | 4000.0       | 18.0         |
| 9  | 2025-01-05   | Bangalore| Amazon   | Laptop     | 1        | 65000.0      | 18.0         |
| 11 | 2025-01-05   | Bangalore| Amazon   | Laptop     | 1        | 65000.0      | 18.0         |
| 12 | 2025-01-05   | Bangalore| Amazon   | Laptop     | 1        | 65000.0      | 18.0         |
| 13 | 2025-01-05   | Bangalore| Amazon   | Laptop     | 1        | 65000.0      | 18.0         |
